# climatecasechart, run surya layout


In [16]:
#@title Import

import re
import os
import json
import time
import pickle
import requests
import numpy as np
import pandas as pd
from tqdm.cli import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool

from IPython.display import display
from shapely.geometry import box

import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from boxly.ops import iou

tqdm.pandas()


# os.chdir('/slow-data/unitednationsclimatehealth/')
os.chdir('/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/')

base_url = 'https://climatecasechart.com/non-us-jurisdiction/' #@param {"type": "string"}
experiment_date = '2025-03-28-run-01' #@param {"type": "string"}

In [ ]:
#@title Run surya (looooong)

case_details = pd.read_pickle(f'data/{experiment_date}-case-details.p3')

# always_run = False #@param {"type": "boolean"}

def run_layout(pdf_path, layout_path):
  os.makedirs(layout_path, exist_ok=True)
  if not os.path.exists(f'{layout_path}/results.json'):
    !CUDA_VISIBLE_DEVICES=1 LAYOUT_BATCH_SIZE=32 surya_layout '{pdf_path}' --output_dir '{layout_path}'


for (slug, link), documents in tqdm(case_details[
    # not already finished
    ~case_details['slug'].apply(
        lambda x: os.path.exists(f'data/{experiment_date}-surya-layout/{x}/results.json'))
    # and actually has documents:
    & (case_details['documents'].apply(lambda x: [x for x in x if x['File'] != '']).str.len() > 0)
].groupby(['slug', 'link'])['documents'], position=0):
  pdf_path = f'data/{experiment_date}-pdf/{slug}/'
  layout_path = f'data/{experiment_date}-surya-layout/{slug}/'

  run_layout(pdf_path, layout_path)

In [17]:
#@title Re-index layout dictionaries to actual filename

for slug in case_details['slug'].unique():
  if not os.path.exists(f'data/{experiment_date}-surya-layout/{slug}/results.json'):
    continue
  # if os.path.exists(f'data/{experiment_date}-surya-layout/{slug}/fixed-results.json'):
  #   continue
  with open(f'data/{experiment_date}-surya-layout/{slug}/results.json') as f:
    layouts = json.load(f)
  fns = [fn for fn in os.listdir(f'data/{experiment_date}-pdf/{slug}/') if fn.endswith('.pdf')]
  if len(layouts) == len(fns):
    layouts = {
        min((fn for fn in fns if fn.startswith(k)), key=len): v for k, v in layouts.items()
    }
    with open(f'data/{experiment_date}-surya-layout/{slug}/fixed-results.json', 'w') as f:
      json.dump(layouts, f)
  else:
    print(slug)